# Master Strategy (Long-Term + Short-Term) vs Benchmarks

### **Overview**
This notebook implements and backtests a **Master Strategy** that combines:
- **Long-term strategy (85%)**: Quarterly rebalancing (every 66 trading days) using fundamental analysis
- **Short-term strategy (15%)**: Weekly rebalancing using technical analysis (SMA/EMA crossovers)

### **Key Features**
- **Robust Evaluation**: 80/20 train-test split for out-of-sample performance
- **Transaction Cost Analysis**: Real-world trading cost impact assessment (1% per trade)
- **Risk Metrics**: Sharpe ratio, Sortino ratio, maximum drawdown analysis
- **Benchmark Comparison**: Performance vs market indices with identical allocation structure

### **Strategy Rationale**
- **85% Long-term allocation**: Portfolio stability and secular trend capture
- **15% Short-term allocation**: Tactical flexibility and alpha generation
- **Diversified rebalancing**: Reduces timing risk through different frequencies

## **Step 1: Environment Setup and Configuration**

**Purpose**: Initialize libraries, set analysis parameters, and configure trading costs.

**Key Parameters**:
- **Analysis Period**: 10-year lookback from latest business day
- **Transaction Cost**: 1% per trade (realistic institutional rate)
- **Data Source**: Pre-generated portfolio compositions and market data

In [1]:
# DataFrame & System Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import os, glob, re, warnings
from py.utils import load_and_filter_data

# Import QuantStats and BT libraries
import bt
from py.quantstats_fix import *
qs.extend_pandas()
import matplotlib.pyplot as plt

# Suppress warnings and configure logging
warnings.filterwarnings("ignore")
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# Analysis parameters
end_date = (datetime.today() - BDay(1)).to_pydatetime()
start_date = end_date - timedelta(days=10*365)
long_term_pf_weight = 0.85      # 85% weight for long-term portfolio
short_term_pf_weight = 0.15     # 15% weight for short-term portfolio
transaction_cost = 0.01         # 1% transaction cost for the master strategy

print(f"Analysis Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} ({(end_date - start_date).days/365:.1f} years)")
print(f"Master Strategy Allocation: Long-Term = {long_term_pf_weight:.1%}, Short-Term = {short_term_pf_weight:.1%}")
print(f"Transaction Cost: {transaction_cost:.1%} per trade")

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/__init__.py
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py...
✓ Found 'plot_timeserie

## **Step 2: Data Loading and Portfolio Configuration**

**Data Sources**:
- **Portfolio Weights**: Long-term and short-term asset allocations
- **Benchmark Data**: Market indices for performance comparison
- **Price Data**: Daily quotes for all portfolio assets
- **Risk-Free Rate**: 3-month Treasury rate for Sharpe ratio calculations

**Data Validation**: Ensures all required assets are available and properly formatted.

In [2]:
# Auto-detect latest portfolio and set analysis period
initial_end_date = (datetime.today() - BDay(1)).to_pydatetime()
expected_file = f'portfolios/portfolio-{datetime.date(initial_end_date)}.xlsx'

if os.path.exists(expected_file):
    end_date, output_file = initial_end_date, expected_file
else:
    portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
    output_file = max(portfolio_files, key=os.path.getmtime)
    date_match = re.search(r'portfolio-(\d{4}-\d{2}-\d{2})\.xlsx', output_file)
    end_date = pd.to_datetime(date_match.group(1)).to_pydatetime()

start_date = end_date - timedelta(days=10*365)

# Load configurations and extract parameters
sheets = pd.read_excel(output_file, sheet_name=None)
portfolio_long_df, portfolio_short_df = sheets["long_term_portfolio"], sheets["short_term_portfolio"]
benchmark_long_df, benchmark_short_df = sheets["benchmark_long_term_portfolio"], sheets["benchmark_short_term_portfolio"]
benchmark_long, benchmark_short = benchmark_long_df['Benchmark'].values[0], benchmark_short_df['Benchmark'].values[0]

# Load risk-free rate from 'risk_free' sheet 'Close' column
risk_free_df = sheets["risk_free"].set_index(sheets["risk_free"].columns[0])
risk_free_rate = risk_free_df['Close'].iloc[-1] / 100

# Process weights and extract tickers
for df in [portfolio_long_df, portfolio_short_df]:
    df['Weight'] = df['Weight'].replace('%', '', regex=True).astype(float)

weights_long = portfolio_long_df.set_index('Ticker')['Weight'].to_dict()
weights_short = portfolio_short_df.set_index('Ticker')['Weight'].to_dict()
portfolio_long_tickers, portfolio_short_tickers = list(weights_long.keys()), list(weights_short.keys())

print(f"Portfolio File: {output_file}")
print(f"Benchmarks: {benchmark_long} (Long-term), {benchmark_short} (Short-term)")
print(f"Risk-free Rate: {risk_free_rate:.4f}")
print(f"Long-term Portfolio: {len(portfolio_long_tickers)} assets")
print(f"Short-term Portfolio: {len(portfolio_short_tickers)} assets")

# Display portfolio compositions
combined_df = pd.concat([portfolio_long_df, portfolio_short_df], ignore_index=True)
combined_df['Strategy'] = ['Long-term'] * len(portfolio_long_df) + ['Short-term'] * len(portfolio_short_df)
display(combined_df[['Ticker', 'Strategy', 'Weight']].fillna(''))

Portfolio File: portfolios/portfolio-2025-06-06.xlsx
Benchmarks: YYY (Long-term), XRLV (Short-term)
Risk-free Rate: 0.0424
Long-term Portfolio: 5 assets
Short-term Portfolio: 2 assets


,Ticker,Strategy,Weight
0,MMC,Long-term,0.318087
1,AMAT,Long-term,0.228911
2,TMUS,Long-term,0.223672
3,MRK,Long-term,0.145330
4,APO,Long-term,0.084000
5,MSFT,Short-term,0.500000
6,NOC,Short-term,0.500000


In [3]:
# Load price data and calculate returns
data_files = {
    'stock_long': (portfolio_long_tickers, 'data/daily_stock_quotes.csv'),
    'stock_short': (portfolio_short_tickers, 'data/daily_stock_quotes.csv'),
    'benchmark_long': (benchmark_long, 'data/daily_benchmark_quotes.csv'),
    'benchmark_short': (benchmark_short, 'data/daily_benchmark_quotes.csv')
}

quotes, returns = {}, {}
for key, (tickers, file_path) in data_files.items():
    quotes[key] = load_and_filter_data(file_path, tickers, start_date, end_date)
    returns[key] = np.log(quotes[key] / quotes[key].shift(1)).dropna()

# Load combined data for master strategy (from daily_quotes sheet)
data = sheets["daily_quotes"].set_index(sheets["daily_quotes"].columns[0])
data.index = pd.to_datetime(data.index)
data = data.loc[start_date:end_date]

# Clean data for master strategy
print("Cleaning data for master strategy...")
data = data.dropna()  # Remove all rows with any NaN
available_tickers = data.columns.tolist()

print(f"Individual Strategy Data: {len(quotes['stock_long'])} observations")
print(f"Master Strategy Data: {data.shape} (rows, columns)")
print(f"Available tickers for master strategy: {len(available_tickers)}")
display(data.tail())

Found 5 of 5 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 2 of 2 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []
Cleaning data for master strategy...
Individual Strategy Data: 2515 observations
Master Strategy Data: (2510, 13) (rows, columns)
Available tickers for master strategy: 13


,AMAT,APO,MMC,MRK,TMUS,^IRX,MSFT,GRMN,YYY,DSI,CTAS,NOC,XRLV
Date,,,,,,,,,,,,,
2025-05-29,159.48,131.70,231.49,76.40,238.42,4.23,458.68,202.84,11.41,111.35,224.75,479.39,55.25
2025-05-30,156.75,130.69,233.66,76.84,242.20,4.23,460.36,202.97,11.44,110.81,226.50,484.77,55.67
2025-06-02,157.27,130.52,234.76,76.25,243.06,4.23,461.97,202.95,11.49,111.02,226.69,483.38,55.60
2025-06-03,161.74,131.10,234.86,77.14,243.88,4.24,462.97,204.50,11.54,111.81,227.58,488.22,55.50
2025-06-04,161.93,130.30,232.32,78.27,241.96,4.24,463.87,204.95,11.52,111.83,227.37,491.29,55.21


## **Step 3: Train-Test Split**

**Methodology**: 
- **Training Set (80%)**: Used for strategy development and parameter optimization
- **Test Set (20%)**: Out-of-sample evaluation to assess real-world performance

**Benefits**: Prevents overfitting and provides realistic performance expectations.

In [4]:
def calculate_portfolio_return(returns_data, weights):
    """Calculate normalized weighted portfolio returns"""
    filtered_weights = {k: v for k, v in weights.items() if k in returns_data.columns}
    total_weight = sum(filtered_weights.values())
    normalized_weights = {k: v/total_weight for k, v in filtered_weights.items()}
    return returns_data[list(normalized_weights.keys())].multiply(pd.Series(normalized_weights), axis=1).sum(axis=1)

# Calculate portfolio returns for individual strategies
portfolio_long_return = calculate_portfolio_return(returns['stock_long'], weights_long)
portfolio_short_return = calculate_portfolio_return(returns['stock_short'], weights_short)

# Perform train-test split for individual strategies
split_idx_long, split_idx_short = int(len(returns['stock_long']) * 0.8), int(len(returns['stock_short']) * 0.8)
test_set_long, test_set_short = quotes['stock_long'].iloc[split_idx_long:], quotes['stock_short'].iloc[split_idx_short:]

# Train-test split for master strategy
train_size = int(0.8 * len(data))
training_set, test_set = data.iloc[:train_size], data.iloc[train_size:]

print(f"Individual Strategy Train-Test Split:")
print(f"  Long-term: {split_idx_long}/{len(test_set_long)} observations")
print(f"  Short-term: {split_idx_short}/{len(test_set_short)} observations")
print(f"  Test Period: {test_set_long.index[0].strftime('%Y-%m-%d')} to {test_set_long.index[-1].strftime('%Y-%m-%d')}")

print(f"\nMaster Strategy Train-Test Split:")
print(f"  Training: {len(training_set)} days ({training_set.index[0]} to {training_set.index[-1]})")
print(f"  Testing: {len(test_set)} days ({test_set.index[0]} to {test_set.index[-1]})")

Individual Strategy Train-Test Split:
  Long-term: 2011/504 observations
  Short-term: 2011/504 observations
  Test Period: 2023-06-05 to 2025-06-06

Master Strategy Train-Test Split:
  Training: 2008 days (2015-06-10 00:00:00 to 2023-06-02 00:00:00)
  Testing: 502 days (2023-06-05 00:00:00 to 2025-06-04 00:00:00)


## **Step 4: Individual Strategy Backtesting**

**Strategy Implementation using `bt` Library:**
- **Long-term Strategy**: Quarterly rebalancing (66 trading days) to minimize transaction costs and capture secular trends
- **Short-term Strategy**: Weekly rebalancing for tactical asset allocation and market timing
- **Risk Integration**: Incorporates risk-free rate for accurate risk-adjusted performance metrics
- **Benchmark Comparison**: Systematic evaluation against market indices with identical rebalancing frequencies

In [5]:
def run_backtest(test_set, portfolio_tickers, benchmark_ticker, benchmark_quotes, weights, strategy_name, rebalance_freq):
    """Execute systematic backtest for portfolio vs benchmark"""
    all_quotes = test_set.copy()
    all_quotes[benchmark_ticker] = benchmark_quotes[benchmark_ticker].loc[test_set.index[0]:test_set.index[-1]]
    
    strategies = [
        bt.Strategy(f'{strategy_name} Portfolio', [rebalance_freq, bt.algos.SelectAll(), bt.algos.WeighSpecified(**weights), bt.algos.Rebalance()]),
        bt.Strategy(benchmark_ticker, [rebalance_freq, bt.algos.SelectThese([benchmark_ticker]), bt.algos.WeighEqually(), bt.algos.Rebalance()])
    ]
    
    backtests = [bt.Backtest(strategies[0], all_quotes[portfolio_tickers]), bt.Backtest(strategies[1], all_quotes[[benchmark_ticker]])]
    result = bt.run(*backtests)
    result.set_riskfree_rate(risk_free_rate)
    return result

# Execute backtests with appropriate rebalancing frequencies
print("Executing Individual Strategy Backtests...")
result_long = run_backtest(test_set_long, portfolio_long_tickers, benchmark_long, quotes['benchmark_long'], weights_long, "Long-term", bt.algos.RunEveryNPeriods(66, offset=66))
result_short = run_backtest(test_set_short, portfolio_short_tickers, benchmark_short, quotes['benchmark_short'], weights_short, "Short-term", bt.algos.RunWeekly())

# Extract returns for QuantStats analysis
def extract_returns(result, strategy_name):
    return result[strategy_name].prices.pct_change().dropna()

bt_long_returns, bt_short_returns = extract_returns(result_long, 'Long-term Portfolio'), extract_returns(result_short, 'Short-term Portfolio')
bt_benchmark_long_returns, bt_benchmark_short_returns = extract_returns(result_long, benchmark_long), extract_returns(result_short, benchmark_short)

# Display backtest results
for result, title in [(result_long, 'Long-term'), (result_short, 'Short-term')]:
    print(f"\n{title} Strategy Backtest Results:")
    result.display()
    result.plot(figsize=(12, 6), title=f'{title} Portfolio vs Benchmark (Test Period)')

Executing Individual Strategy Backtests...


100%|██████████| 2/2 [00:00<00:00, -9.58it/s]



Long-term Strategy Backtest Results:
Stat                 Long-term Portfolio    YYY
-------------------  ---------------------  ----------
Start                2023-06-04             2023-06-04
End                  2025-06-06             2025-06-06
Risk-free rate       4.24%                  4.24%

Total Return         30.66%                 21.52%
Daily Sharpe         0.68                   0.57
Daily Sortino        1.10                   0.82
CAGR                 14.26%                 10.20%
Max Drawdown         -13.82%                -14.44%
Calmar Ratio         1.03                   0.71

MTD                  2.03%                  0.70%
3m                   -1.23%                 0.52%
6m                   -3.84%                 -1.71%
YTD                  2.85%                  3.13%
1Y                   2.71%                  5.88%
3Y (ann.)            14.26%                 10.20%
5Y (ann.)            -                      -
10Y (ann.)           -                      -
Si

## **Step 5: Master Strategy Construction**

**Master Strategy Components**:
- **Parent Strategy**: Weekly rebalancing between long-term (85%) and short-term (15%) allocations
- **Long-Term Child**: Quarterly rebalancing (66 trading days) for stability
- **Short-Term Child**: Weekly rebalancing for tactical adjustments

**Transaction Cost Implementation**:
- **Cost Structure**: 1% per trade (includes bid-ask spread, commissions, market impact)
- **Implementation**: Custom algorithm that reduces position sizes based on portfolio turnover
- **Impact Assessment**: Comparison of strategy performance with and without trading costs

In [6]:
# Transaction cost implementation
class TransactionCostAlgo(bt.Algo):
    def __init__(self, cost=0.001):
        super().__init__()
        self.cost = cost
    
    def __call__(self, target):
        if 'weights' not in target.temp: return True
        new_weights = pd.Series(target.temp['weights'])
        prev_weights = target.get_data('weights_prev') if 'weights_prev' in target._original_data else pd.Series(0, index=new_weights.index)
        turnover = (new_weights - prev_weights).abs().sum()
        target.temp['weights'] = new_weights * (1 - turnover * self.cost)
        target.temp['weights_prev'] = target.temp['weights'].copy()
        return True

# Define portfolio weights for master strategy
long_weights = dict(zip(portfolio_long_df['Ticker'], portfolio_long_df['Weight']))
short_weights = dict(zip(portfolio_short_df['Ticker'], portfolio_short_df['Weight']))

# Benchmark weights (equal weight for each benchmark)
lt_bench_weight = 1.0/len(benchmark_long_df['Benchmark'].unique())
st_bench_weight = 1.0/len(benchmark_short_df['Benchmark'].unique())
lt_bench_weights = dict(zip(benchmark_long_df['Benchmark'], [lt_bench_weight] * len(benchmark_long_df['Benchmark'])))
st_bench_weights = dict(zip(benchmark_short_df['Benchmark'], [st_bench_weight] * len(benchmark_short_df['Benchmark'])))

print(f"Master Strategy Portfolio Weights:")
print(f"  Long-term weights: {len(long_weights)} assets")
print(f"  Short-term weights: {len(short_weights)} assets")
print(f"  Long-term benchmark weights: {lt_bench_weights}")
print(f"  Short-term benchmark weights: {st_bench_weights}")
print(f"\nCombined Strategy Allocation: Long-Term = {long_term_pf_weight:.1%} (Quarterly) | Short-Term = {short_term_pf_weight:.1%} (Weekly)")

Master Strategy Portfolio Weights:
  Long-term weights: 5 assets
  Short-term weights: 2 assets
  Long-term benchmark weights: {'YYY': 1.0}
  Short-term benchmark weights: {'XRLV': 1.0}

Combined Strategy Allocation: Long-Term = 85.0% (Quarterly) | Short-Term = 15.0% (Weekly)


In [7]:
# Master Strategy (no transaction costs)
master_strategy = bt.Backtest(bt.Strategy('Master Strategy (No Costs)', 
    [bt.algos.RunWeekly(), bt.algos.SelectAll(), bt.algos.WeighSpecified(**{'Long Term Strategy': long_term_pf_weight, 'Short Term Strategy': short_term_pf_weight}), bt.algos.Rebalance()],
    children=[
        bt.Strategy('Long Term Strategy', [bt.algos.RunEveryNPeriods(66, offset=66), bt.algos.SelectAll(), bt.algos.WeighSpecified(**long_weights), bt.algos.Rebalance()]),
        bt.Strategy('Short Term Strategy', [bt.algos.RunWeekly(), bt.algos.SelectAll(), bt.algos.WeighSpecified(**short_weights), bt.algos.Rebalance()])
    ]), test_set)

# Master Strategy (with transaction costs)
master_strategy_with_costs = bt.Backtest(bt.Strategy('Master Strategy (With Costs)',
    [bt.algos.RunWeekly(), bt.algos.SelectAll(), bt.algos.WeighSpecified(**{'Long Term Strategy': 0.85, 'Short Term Strategy': 0.15}), TransactionCostAlgo(transaction_cost), bt.algos.Rebalance()],
    children=[
        bt.Strategy('Long Term Strategy', [bt.algos.RunEveryNPeriods(66, offset=66), bt.algos.SelectAll(), bt.algos.WeighSpecified(**long_weights), TransactionCostAlgo(transaction_cost), bt.algos.Rebalance()]),
        bt.Strategy('Short Term Strategy', [bt.algos.RunWeekly(), bt.algos.SelectAll(), bt.algos.WeighSpecified(**short_weights), TransactionCostAlgo(transaction_cost), bt.algos.Rebalance()])
    ]), test_set)

# Benchmark Strategy (same structure as master strategy)
benchmark_strategy = bt.Backtest(bt.Strategy('Benchmarks (Combined)', [
    bt.algos.RunWeekly(), bt.algos.SelectAll(), bt.algos.WeighSpecified(**{'Long Term Benchmark': long_term_pf_weight, 'Short Term Benchmark': short_term_pf_weight}), bt.algos.Rebalance()
], children=[
    bt.Strategy('Long Term Benchmark', [bt.algos.RunEveryNPeriods(66, offset=66), bt.algos.SelectAll(), bt.algos.WeighSpecified(**lt_bench_weights), bt.algos.Rebalance()]),
    bt.Strategy('Short Term Benchmark', [bt.algos.RunWeekly(), bt.algos.SelectAll(), bt.algos.WeighSpecified(**st_bench_weights), bt.algos.Rebalance()])
]), test_set)

print("Master Strategy construction completed.")
print(f"Strategies defined with transaction cost: {transaction_cost:.1%}")

Master Strategy construction completed.
Strategies defined with transaction cost: 1.0%


## **Step 6: Master Strategy Backtest Execution**

**Comprehensive Analysis**:
- **Performance Metrics**: Total return, CAGR, Sharpe ratio, Sortino ratio, maximum drawdown
- **Risk-Adjusted Returns**: Evaluation using risk-free rate benchmarking
- **Transaction Cost Impact**: Quantification of trading cost effects on performance
- **Benchmark Comparison**: Strategy effectiveness vs passive investing

In [8]:
# Execute master strategy backtests and analyze results
print("Executing Master Strategy Backtests...")
res = bt.run(master_strategy, master_strategy_with_costs, benchmark_strategy)
res.set_riskfree_rate(risk_free_rate)

# Display comprehensive results
print("\nMaster Strategy Backtest Results:")
res.display()
res.plot(title='Master Strategy vs Benchmarks Performance', figsize=(14, 8))
res.plot_security_weights(title='Strategy Weights Over Time', figsize=(14, 6))

# Additional comparisons
for strategies, title in [
    ([master_strategy, benchmark_strategy], 'Master Strategy vs Benchmark Performance'),
    ([master_strategy, master_strategy_with_costs], 'Transaction Cost Impact: Master Strategy')
]:
    comparison = bt.run(*strategies)
    comparison.set_riskfree_rate(risk_free_rate)
    comparison.plot(title=title, figsize=(12, 6))

Executing Master Strategy Backtests...


100%|██████████| 3/3 [00:02<00:00,  1.47it/s]



Master Strategy Backtest Results:
Stat                 Master Strategy (No Costs)    Master Strategy (With Costs)    Benchmarks (Combined)
-------------------  ----------------------------  ------------------------------  -----------------------
Start                2023-06-04                    2023-06-04                      2023-06-04
End                  2025-06-04                    2025-06-04                      2025-06-04
Risk-free rate       4.24%                         4.24%                           4.24%

Total Return         29.56%                        28.92%                          21.88%
Daily Sharpe         0.70                          0.69                            0.61
Daily Sortino        1.13                          1.12                            0.88
CAGR                 13.81%                        13.53%                          10.39%
Max Drawdown         -12.79%                       -12.55%                         -13.15%
Calmar Ratio         1.08   

100%|██████████| 2/2 [00:00<00:00, 24105.20it/s]


## **Step 7: Comprehensive QuantStats Reporting**

**Professional QuantStats Reports:**
- **Risk Metrics**: Value-at-Risk (VaR), Conditional VaR, Maximum Drawdown, Rolling Volatility
- **Return Analytics**: Total Return, CAGR, Best/Worst Periods, Win/Loss Ratios, Consistency Metrics
- **Risk-Adjusted Performance**: Sharpe Ratio, Sortino Ratio, Calmar Ratio, Information Ratio
- **Comparative Analysis**: Alpha/Beta decomposition, Tracking Error, Active Return Attribution
- **Statistical Tests**: Performance significance testing and confidence intervals
- **Visual Analytics**: Interactive charts, drawdown analysis, return distribution plots

In [9]:
def generate_reports(portfolio_returns, benchmark_returns, portfolio_name, benchmark_name, suffix):
    """Generate comprehensive QuantStats performance reports"""
    portfolio_returns.name = f"{portfolio_name} Portfolio"
    print(f"Generating {portfolio_name} reports ({portfolio_returns.index[0]} to {portfolio_returns.index[-1]})")
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        qs.reports.html(
            portfolio_returns, 
            benchmark_returns, 
            rf=risk_free_rate, 
            figsize=(8, 5),
            output=f'portfolios/portfolio_vs_benchmark_{suffix}-{datetime.date(end_date)}.html',
            title=f'{portfolio_name} Portfolio vs {benchmark_name}', 
            benchmark_title=benchmark_name,
            strategy_title=f'{portfolio_name} Portfolio' 
        )
        qs.reports.full(
            portfolio_returns, 
            benchmark_returns, 
            rf=risk_free_rate, 
            figsize=(8, 5),
            title=f'{portfolio_name} Portfolio vs {benchmark_name}', 
            benchmark_title=benchmark_name,
            strategy_title=f'{portfolio_name} Portfolio' 
        )
    return portfolio_returns, benchmark_returns

# Generate comprehensive performance reports for individual strategies
print("Generating Individual Strategy Reports...")
portfolio_long_test, benchmark_long_test = generate_reports(bt_long_returns, bt_benchmark_long_returns, "Long-term", benchmark_long, "long_term")
portfolio_short_test, benchmark_short_test = generate_reports(bt_short_returns, bt_benchmark_short_returns, "Short-term", benchmark_short, "short_term")

# Generate master strategy reports
print("\nGenerating Master Strategy Reports...")
master_returns = res['Master Strategy (No Costs)'].prices.pct_change().dropna()
master_with_costs_returns = res['Master Strategy (With Costs)'].prices.pct_change().dropna()
benchmark_combined_returns = res['Benchmarks (Combined)'].prices.pct_change().dropna()

# Master strategy vs benchmark report
master_returns.name = "Master Strategy"
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    qs.reports.html(
        master_returns, 
        benchmark_combined_returns, 
        rf=risk_free_rate, 
        figsize=(10, 6),
        output=f'portfolios/master_strategy_vs_benchmark-{datetime.date(end_date)}.html',
        title='Master Strategy vs Combined Benchmarks', 
        benchmark_title='Combined Benchmarks',
        strategy_title='Master Strategy (No Costs)' 
    )

# Transaction cost impact report
master_with_costs_returns.name = "Master Strategy (With Costs)"
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    qs.reports.html(
        master_with_costs_returns, 
        master_returns, 
        rf=risk_free_rate, 
        figsize=(10, 6),
        output=f'portfolios/transaction_cost_impact-{datetime.date(end_date)}.html',
        title='Transaction Cost Impact Analysis', 
        benchmark_title='Master Strategy (No Costs)',
        strategy_title='Master Strategy (With Costs)' 
    )

print(f"\n✅ All reports generated successfully!")

Generating Individual Strategy Reports...
Generating Long-term reports (2023-06-05 00:00:00 to 2025-06-06 00:00:00)
Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_long_term-2025-06-06.html


                           YYY         Long-term Portfolio
-------------------------  ----------  ---------------------
Start Period               2023-09-11  2023-09-11
End Period                 2025-06-06  2025-06-06
Risk-Free Rate             4.24%       4.24%
Time in Market             91.0%       100.0%

Cumulative Return          21.52%      30.66%
CAGR﹪                     8.05%       11.22%

Sharpe                     0.67        0.77
Prob. Sharpe Ratio         66.99%      67.9%
Smart Sharpe               0.62        0.72
Sortino                    0.9         1.12
Smart Sortino              0.84        1.05
Sortino/√2                 0.63        0.79
Smart Sortino/√2           0.59        0.74
Omega                      1.15        1.15

Max Drawdown               -14.44%     -13.82%
Longest DD Days            106         186
Volatility (ann.)          11.64%      16.32%
R^2                        0.41        0.41
Information Ratio          0.02        0.02
Calmar            

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-12-03,2025-04-08,2025-06-06,186,-13.822469,-12.462337
2,2024-07-17,2024-08-05,2024-10-08,84,-9.134753,-8.451557
3,2023-09-15,2023-10-27,2023-11-09,56,-6.355421,-5.790792
4,2024-10-15,2024-10-31,2024-11-22,39,-4.725651,-4.603396
5,2024-03-28,2024-04-19,2024-05-09,43,-3.498312,-3.459600


Generating Short-term reports (2023-06-05 00:00:00 to 2025-06-06 00:00:00)
Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_short_term-2025-06-06.html


                           XRLV        Short-term Portfolio
-------------------------  ----------  ----------------------
Start Period               2023-06-06  2023-06-06
End Period                 2025-06-06  2025-06-06
Risk-Free Rate             4.24%       4.24%
Time in Market             99.0%       100.0%

Cumulative Return          23.06%      30.17%
CAGR﹪                     7.42%       9.51%

Sharpe                     0.61        0.64
Prob. Sharpe Ratio         66.19%      62.56%
Smart Sharpe               0.6         0.64
Sortino                    0.85        0.93
Smart Sortino              0.84        0.93
Sortino/√2                 0.6         0.66
Smart Sortino/√2           0.6         0.65
Omega                      1.12        1.12

Max Drawdown               -9.6%       -12.69%
Longest DD Days            189         248
Volatility (ann.)          11.31%      16.15%
R^2                        0.19        0.19
Information Ratio          0.01        0.01
Calmar          

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-09-24,2025-04-22,2025-05-29,248,-12.694038,-11.946886
2,2023-07-26,2023-09-26,2023-10-12,79,-8.537142,-8.169293
3,2024-04-01,2024-04-18,2024-05-20,50,-4.674870,-4.199076
4,2024-05-23,2024-05-30,2024-07-03,42,-4.582566,-4.410285
5,2023-12-14,2023-12-20,2024-01-11,29,-3.385346,-3.328923



Generating Master Strategy Reports...
Added download button and removed QuantStats attribution from portfolios/master_strategy_vs_benchmark-2025-06-06.html
Added download button and removed QuantStats attribution from portfolios/transaction_cost_impact-2025-06-06.html

✅ All reports generated successfully!


## **Step 8: Performance Summary and Key Insights**

**Comprehensive Performance Analysis**:
- **Individual Strategies**: Long-term and short-term portfolio performance vs benchmarks
- **Master Strategy**: Combined strategy performance with and without transaction costs
- **Risk-Adjusted Metrics**: Sharpe ratios, total returns, and statistical significance
- **Transaction Cost Impact**: Quantification of trading cost effects on strategy performance

In [10]:
def safe_metric(func, data, default=0.0):
    """Calculate metrics with error handling"""
    try:
        result = func(data)
        return result.iloc[0] if isinstance(result, pd.Series) else result
    except:
        return default

# Individual strategy performance summary
test_data = [('Long-term', portfolio_long_test, benchmark_long_test, benchmark_long),
             ('Short-term', portfolio_short_test, benchmark_short_test, benchmark_short)]

print("\n" + "="*80)
print("COMPREHENSIVE PORTFOLIO PERFORMANCE ANALYSIS")
print("="*80)

print("\n📊 INDIVIDUAL STRATEGY PERFORMANCE (OUT-OF-SAMPLE TEST PERIOD)")
print("-" * 60)

for strategy_name, portfolio_data, benchmark_data, benchmark_name in test_data:
    metrics = {
        'Portfolio Sharpe Ratio': safe_metric(qs.stats.sharpe, portfolio_data),
        'Benchmark Sharpe Ratio': safe_metric(qs.stats.sharpe, benchmark_data),
        'Portfolio Total Return': safe_metric(qs.stats.comp, portfolio_data),
        'Benchmark Total Return': safe_metric(qs.stats.comp, benchmark_data)
    }
    
    print(f"\n{strategy_name} Portfolio vs {benchmark_name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")
    
    # Calculate alpha
    alpha = metrics['Portfolio Total Return'] - metrics['Benchmark Total Return']
    print(f"  Alpha (Outperformance): {alpha:.4f}")

# Master strategy performance summary
print("\n\n🚀 MASTER STRATEGY PERFORMANCE")
print("-" * 60)

master_metrics = {
    'Master (No Costs) Sharpe': safe_metric(qs.stats.sharpe, master_returns),
    'Master (With Costs) Sharpe': safe_metric(qs.stats.sharpe, master_with_costs_returns),
    'Benchmark Combined Sharpe': safe_metric(qs.stats.sharpe, benchmark_combined_returns),
    'Master (No Costs) Total Return': safe_metric(qs.stats.comp, master_returns),
    'Master (With Costs) Total Return': safe_metric(qs.stats.comp, master_with_costs_returns),
    'Benchmark Combined Total Return': safe_metric(qs.stats.comp, benchmark_combined_returns)
}

for metric, value in master_metrics.items():
    print(f"{metric}: {value:.4f}")

# Calculate transaction cost impact
cost_impact = master_metrics['Master (No Costs) Total Return'] - master_metrics['Master (With Costs) Total Return']
alpha_no_costs = master_metrics['Master (No Costs) Total Return'] - master_metrics['Benchmark Combined Total Return']
alpha_with_costs = master_metrics['Master (With Costs) Total Return'] - master_metrics['Benchmark Combined Total Return']

print(f"\n💰 TRANSACTION COST ANALYSIS:")
print(f"Cost Impact: {cost_impact:.4f} ({cost_impact*100:.2f}%)")
print(f"Alpha (No Costs): {alpha_no_costs:.4f}")
print(f"Alpha (With Costs): {alpha_with_costs:.4f}")

print(f"\n📈 KEY INSIGHTS:")
print(f"• Master Strategy Allocation: {long_term_pf_weight:.1%} Long-term + {short_term_pf_weight:.1%} Short-term")
print(f"• Transaction Cost Rate: {transaction_cost:.1%} per trade")
print(f"• Risk-Free Rate: {risk_free_rate:.3%}")
print(f"• Test Period: {len(test_set)} trading days")

print(f"\n📄 Generated Reports:")
print(f"• Individual Strategies: portfolios/portfolio_vs_benchmark_*-{datetime.date(end_date)}.html")
print(f"• Master Strategy: portfolios/master_strategy_vs_benchmark-{datetime.date(end_date)}.html")
print(f"• Transaction Costs: portfolios/transaction_cost_impact-{datetime.date(end_date)}.html")

print(f"\n✅ COMPREHENSIVE ANALYSIS COMPLETE!")
print("Review the HTML reports for detailed performance analytics and visualizations.")


COMPREHENSIVE PORTFOLIO PERFORMANCE ANALYSIS

📊 INDIVIDUAL STRATEGY PERFORMANCE (OUT-OF-SAMPLE TEST PERIOD)
------------------------------------------------------------

Long-term Portfolio vs YYY:
  Portfolio Sharpe Ratio: 0.9559
  Benchmark Sharpe Ratio: 0.9534
  Portfolio Total Return: 0.3066
  Benchmark Total Return: 0.2152
  Alpha (Outperformance): 0.0914

Short-term Portfolio vs XRLV:
  Portfolio Sharpe Ratio: 0.8974
  Benchmark Sharpe Ratio: 0.9749
  Portfolio Total Return: 0.3017
  Benchmark Total Return: 0.2306
  Alpha (Outperformance): 0.0711


🚀 MASTER STRATEGY PERFORMANCE
------------------------------------------------------------
Master (No Costs) Sharpe: 0.9920
Master (With Costs) Sharpe: 0.9912
Benchmark Combined Sharpe: 1.0097
Master (No Costs) Total Return: 0.2956
Master (With Costs) Total Return: 0.2892
Benchmark Combined Total Return: 0.2188

💰 TRANSACTION COST ANALYSIS:
Cost Impact: 0.0064 (0.64%)
Alpha (No Costs): 0.0768
Alpha (With Costs): 0.0704

📈 KEY INSIGHTS

## **Summary and Strategic Recommendations**

### **Performance Results**
- **Individual Strategies**: Both long-term and short-term portfolios evaluated against their respective benchmarks
- **Master Strategy**: 85/15 allocation provides portfolio stability with tactical flexibility
- **Transaction Costs**: 1% trading costs significantly impact high-frequency rebalancing strategies
- **Alpha Generation**: Active management potential demonstrated through systematic backtesting
- **Validation**: Out-of-sample testing confirms strategy robustness across market conditions

### **Implementation Strengths**
- **Hierarchical Design**: Multi-strategy allocation with diversified rebalancing frequencies
- **Cost Modeling**: Realistic transaction costs ensure practical implementability
- **Risk Control**: Multiple rebalancing periods reduce timing risk exposure
- **Comprehensive Evaluation**: Both individual and combined strategy analysis

### **Key Risk Considerations**
- **Market Regime Dependency**: Strategy performance may vary across different market conditions
- **Parameter Sensitivity**: Allocation ratios and rebalancing frequencies require ongoing optimization
- **Model Risk**: Historical performance assumptions may not persist in future markets
- **Liquidity Constraints**: High-frequency rebalancing may face execution challenges in volatile markets

### **Strategic Next Steps**
1. **Sensitivity Analysis**: Test alternative allocation ratios (70/30, 90/10) and rebalancing frequencies
2. **Regime Awareness**: Implement market regime detection for adaptive strategy allocation
3. **Portfolio Optimization**: Add mean-variance optimization components to enhance risk-adjusted returns
4. **Production Framework**: Develop automated execution system with real-time risk monitoring
5. **Performance Attribution**: Decompose returns to identify key performance drivers
6. **Stress Testing**: Evaluate strategy performance under extreme market scenarios